In [9]:
import pandas as pd
from scipy.stats import mannwhitneyu
import numpy as np
import glob
import os

In [15]:
my_df = pd.read_csv("/mnt/c/Users/donna/Downloads/Thesis/rankjes/abscor_ranks_per_feature/cov_spread_rank_results.csv")

In [13]:
csv_dir = '/mnt/c/Users/donna/Downloads/Thesis/rankjes/paper_ranks'  # Replace with actual path
csv_files = glob.glob(os.path.join(csv_dir, "*.csv"))
df_list = [pd.read_csv(file) for file in csv_files]
combined_df = pd.concat(df_list, ignore_index=True)
combined_df.to_csv("/mnt/c/Users/donna/Downloads/Thesis/rankjes/paper_ranks/combined_ranks.csv")

In [16]:
paper_df = pd.read_csv("/mnt/c/Users/donna/Downloads/Thesis/rankjes/paper_ranks/combined_ranks.csv")

In [21]:
paper_df["tissue"] = paper_df["tissue"].str.lower()


In [24]:
results = []
for tissue in sorted(set(my_df["tissue"]).intersection(paper_df["tissue"])):
    m_ranks = my_df[my_df["tissue"] == tissue]["rank"].values
    p_ranks = paper_df[paper_df["tissue"] == tissue]["rank"].values

    if len(m_ranks) >= 3 and len(p_ranks) >= 3:  # ensure enough data points
        stat, p = mannwhitneyu(m_ranks, p_ranks, alternative="two-sided")
        results.append({
            "tissue": tissue,
            "my_median_rank": np.median(m_ranks),
            "paper_median_rank": np.median(p_ranks),
            "my_mean_ranks": np.mean(m_ranks),
            "paper_mean_ranks": np.mean(p_ranks),
            "p_value": p
        })

In [25]:
results_df = pd.DataFrame(results)
results_df.sort_values("p_value", inplace=True)
print(results_df.head())

             tissue  my_median_rank  paper_median_rank  my_mean_ranks  \
3               eye           168.0              253.0     164.889881   
10       lymph_node           152.0               81.0     149.647794   
18           spleen           148.0               92.0     144.892500   
7   large_intestine           124.0              199.0     137.401136   
17  small_intestine           145.0              211.0     151.135577   

    paper_mean_ranks        p_value  
3         223.042053  4.615888e-122  
10        104.179272   4.224585e-56  
18        101.273304   1.379876e-40  
7         185.166076   8.598645e-39  
17        192.582751   5.663814e-36  


In [27]:
results_df.to_csv("/mnt/c/Users/donna/Downloads/Thesis/rankjes/minevspaper_whitneyU.csv")